In [ ]:
#| default_exp utils_email

# email utils
> Email sending utilities using markdown_merge for clean, maintainable email templates

In [ ]:
#| export

from __future__ import annotations
from typing import Optional, List, Dict, Any
import os
import logging
from pathlib import Path

logger = logging.getLogger(__name__)

In [ ]:
from nbdev.showdoc import show_doc

## SMTP Configuration

Configure SMTP settings from environment variables. This works with any SMTP provider including Azure Communication Service via App Registration, SendGrid, AWS SES, Gmail, etc.

In [ ]:
#| export
def get_smtp_config() -> Dict[str, Any]:
    """Get SMTP configuration from environment variables
    
    Returns:
        Dict with SMTP configuration for markdown_merge
        
    Environment Variables Required:
        SMTP_HOST: SMTP server hostname
        SMTP_PORT: SMTP server port (default: 587)
        SMTP_USER: SMTP authentication username
        SMTP_PASSWORD: SMTP authentication password
        SMTP_MAIL_FROM: Sender email address
        
    Environment Variables Optional:
        SMTP_STARTTLS: Use STARTTLS encryption - True/False (default: True)
        SMTP_SSL: Use SSL instead of STARTTLS - True/False (default: False)
        
    Configuration Examples:
        # Azure Communication Service SMTP
        SMTP_HOST=smtp.azurecomm.net
        SMTP_PORT=587
        SMTP_USER=emailmanager
        SMTP_PASSWORD=<app-registration-token>
        SMTP_MAIL_FROM=DoNotReply@finxplorer.com
        SMTP_STARTTLS=True
        SMTP_SSL=False
    
        
    Raises:
        ValueError: If required environment variables are missing
    """
    smtp_host = os.getenv('SMTP_HOST')
    smtp_port = int(os.getenv('SMTP_PORT', '587'))
    smtp_user = os.getenv('SMTP_USER')
    smtp_pass = os.getenv('SMTP_PASSWORD')
    from_email = os.getenv('SMTP_MAIL_FROM')
    
    # SSL and TLS settings - check string and boolean values
    smtp_ssl = os.getenv('SMTP_SSL', 'False')
    use_ssl = smtp_ssl.lower() in ('true', '1', 'yes') if isinstance(smtp_ssl, str) else bool(smtp_ssl)
    
    smtp_starttls = os.getenv('SMTP_STARTTLS', 'True')
    use_tls = smtp_starttls.lower() in ('true', '1', 'yes') if isinstance(smtp_starttls, str) else bool(smtp_starttls)
    
    # If SSL is enabled, disable TLS
    if use_ssl:
        use_tls = False
    
    # Validate required fields
    if not all([smtp_host, smtp_user, smtp_pass, from_email]):
        raise ValueError(
            "SMTP_HOST, SMTP_USER, SMTP_PASSWORD, and SMTP_MAIL_FROM "
            "environment variables are required"
        )
    
    return {
        'host': smtp_host,
        'port': smtp_port,
        'user': smtp_user,
        'password': smtp_pass,
        'from_email': from_email,
        'from_name': '',  # Optional, can be added as SMTP_FROM_NAME if needed
        'use_ssl': use_ssl,
        'use_tls': use_tls
    }

In [ ]:
show_doc(get_smtp_config)

## Template Management

Load markdown email templates from the templates directory.

In [ ]:
#| export
def get_template_path(template_name: str) -> Path:
    """Get path to email template file
    
    Args:
        template_name: Name of template (e.g. 'welcome', 'invitation', 'password_reset')
    
    Returns:
        Path to template file
    """
    # Get the package directory
    package_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
    template_path = package_dir / 'templates' / f'{template_name}.md'
    
    if not template_path.exists():
        raise FileNotFoundError(f"Template not found: {template_path}")
    
    return template_path


def load_template(template_name: str) -> str:
    """Load markdown email template
    
    Args:
        template_name: Name of template (e.g. 'welcome', 'invitation', 'password_reset')
    
    Returns:
        Template content as string
    """
    template_path = get_template_path(template_name)
    return template_path.read_text(encoding='utf-8')

In [ ]:
show_doc(load_template)

In [ ]:
show_doc(get_template_path)

## Email Sending Functions

Send emails using markdown_merge with markdown templates.

In [ ]:
#| export
def send_email(
    to_email: str,
    to_name: str,
    subject: str,
    template_name: str,
    template_vars: Dict[str, str],
    test: bool = False,
    smtp_config: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """Send an email using a markdown template
    
    Args:
        to_email: Recipient email address
        to_name: Recipient name
        subject: Email subject
        template_name: Name of markdown template (e.g. 'welcome', 'invitation')
        template_vars: Dictionary of variables to substitute in template
        test: If True, print email instead of sending (default: False)
        smtp_config: SMTP configuration dict (if None, loads from environment)
    
    Returns:
        Dict with status and details
    
    Example:
        send_email(
            to_email='user@example.com',
            to_name='John Doe',
            subject='Welcome!',
            template_name='welcome',
            template_vars={
                'user_name': 'John',
                'tenant_name': 'Acme Corp',
                'dashboard_url': 'https://app.example.com/dashboard',
                'to_email': 'user@example.com'
            }
        )
    """
    try:
        from markdown_merge import MarkdownMerge, get_addr
    except ImportError:
        raise ImportError(
            "markdown_merge not installed. Install with: pip install markdown_merge"
        )
    
    # Load SMTP config
    if smtp_config is None:
        config = get_smtp_config()
    else:
        config = smtp_config
    
    # Prepare SMTP config for markdown_merge
    mm_smtp_cfg = {
        'host': config['host'],
        'port': config['port'],
        'user': config['user'],
        'password': config['password'],
        'use_ssl': config.get('use_ssl', False),
        'use_tls': config.get('use_tls', True)
    }
    
    # Load template
    template = load_template(template_name)
    
    # Prepare addresses
    from_addr = get_addr(config['from_email'], config.get('from_name', ''))
    to_addr = get_addr(to_email, to_name)
    
    # Create MarkdownMerge instance
    mm = MarkdownMerge(
        [to_addr],
        from_addr,
        subject,
        template,
        smtp_cfg=mm_smtp_cfg,
        inserts=[template_vars],
        test=test
    )
    
    # Send email
    try:
        mm.send_msgs()
        
        if test:
            logger.info(f"Test email printed for {to_email}")
            return {
                "status": "test",
                "to_email": to_email,
                "message": "Email printed (test mode)"
            }
        else:
            logger.info(f"Email sent successfully to {to_email}")
            return {
                "status": "success",
                "to_email": to_email,
                "subject": subject
            }
    except Exception as e:
        logger.error(f"Failed to send email to {to_email}: {str(e)}")
        return {
            "status": "error",
            "to_email": to_email,
            "error": str(e)
        }

In [ ]:
show_doc(send_email)

## Batch Email Sending

Send the same email to multiple recipients with personalized content.

In [ ]:
#| export
def send_batch_emails(
    recipients: List[Dict[str, str]],
    subject: str,
    template_name: str,
    template_vars_list: List[Dict[str, str]],
    test: bool = False,
    pause: float = 0.2,
    smtp_config: Optional[Dict[str, Any]] = None
) -> List[Dict[str, Any]]:
    """Send personalized emails to multiple recipients
    
    Args:
        recipients: List of dicts with 'email' and 'name' keys
        subject: Email subject (same for all recipients)
        template_name: Name of markdown template
        template_vars_list: List of variable dicts (one per recipient)
        test: If True, print emails instead of sending
        pause: Seconds to pause between sends (default: 0.2)
        smtp_config: SMTP configuration dict (if None, loads from environment)
    
    Returns:
        List of result dicts
    
    Example:
        send_batch_emails(
            recipients=[
                {'email': 'user1@example.com', 'name': 'Alice'},
                {'email': 'user2@example.com', 'name': 'Bob'}
            ],
            subject='Welcome!',
            template_name='welcome',
            template_vars_list=[
                {'user_name': 'Alice', 'tenant_name': 'Acme', ...},
                {'user_name': 'Bob', 'tenant_name': 'Acme', ...}
            ]
        )
    """
    try:
        from markdown_merge import MarkdownMerge, get_addr
    except ImportError:
        raise ImportError(
            "markdown_merge not installed. Install with: pip install markdown_merge"
        )
    
    # Load SMTP config
    if smtp_config is None:
        config = get_smtp_config()
    else:
        config = smtp_config
    
    # Prepare SMTP config for markdown_merge
    mm_smtp_cfg = {
        'host': config['host'],
        'port': config['port'],
        'user': config['user'],
        'password': config['password'],
        'use_ssl': config.get('use_ssl', False),
        'use_tls': config.get('use_tls', True)
    }
    
    # Load template
    template = load_template(template_name)
    
    # Prepare addresses
    from_addr = get_addr(config['from_email'], config.get('from_name', ''))
    to_addrs = [get_addr(r['email'], r['name']) for r in recipients]
    
    # Create MarkdownMerge instance
    mm = MarkdownMerge(
        to_addrs,
        from_addr,
        subject,
        template,
        smtp_cfg=mm_smtp_cfg,
        inserts=template_vars_list,
        test=test
    )
    
    # Send emails
    results = []
    try:
        mm.send_msgs(pause=pause)
        
        for recipient in recipients:
            if test:
                results.append({
                    "status": "test",
                    "to_email": recipient['email'],
                    "message": "Email printed (test mode)"
                })
            else:
                results.append({
                    "status": "success",
                    "to_email": recipient['email'],
                    "subject": subject
                })
        
        logger.info(f"Sent {len(recipients)} emails successfully")
    except Exception as e:
        logger.error(f"Batch send failed: {str(e)}")
        for recipient in recipients:
            results.append({
                "status": "error",
                "to_email": recipient['email'],
                "error": str(e)
            })
    
    return results

In [ ]:
show_doc(send_batch_emails)

## Convenience Template Functions

High-level functions for common email types with pre-defined templates.

In [ ]:
#| export
def send_welcome_email(
    to_email: str,
    to_name: str,
    user_name: str,
    tenant_name: str,
    dashboard_url: str,
    test: bool = False
) -> Dict[str, Any]:
    """Send welcome email to new user
    
    Args:
        to_email: User's email address
        to_name: User's full name for email header
        user_name: User's display name for template
        tenant_name: Tenant name
        dashboard_url: URL to dashboard
        test: If True, print email instead of sending
    
    Returns:
        Result dict from send operation
    """
    return send_email(
        to_email=to_email,
        to_name=to_name,
        subject=f"Welcome to {tenant_name}!",
        template_name='welcome',
        template_vars={
            'user_name': user_name,
            'tenant_name': tenant_name,
            'dashboard_url': dashboard_url,
            'to_email': to_email
        },
        test=test
    )

In [ ]:
show_doc(send_welcome_email)

In [ ]:
#| export
def send_invitation_email(
    to_email: str,
    to_name: str,
    inviter_name: str,
    tenant_name: str,
    invitation_url: str,
    test: bool = False
) -> Dict[str, Any]:
    """Send invitation email to join tenant
    
    Args:
        to_email: Invitee's email address
        to_name: Invitee's name for email header
        inviter_name: Name of person sending invitation
        tenant_name: Tenant name
        invitation_url: URL with invitation token
        test: If True, print email instead of sending
    
    Returns:
        Result dict from send operation
    """
    return send_email(
        to_email=to_email,
        to_name=to_name,
        subject=f"{inviter_name} invited you to join {tenant_name}",
        template_name='invitation',
        template_vars={
            'inviter_name': inviter_name,
            'tenant_name': tenant_name,
            'invitation_url': invitation_url,
            'to_email': to_email
        },
        test=test
    )

In [ ]:
show_doc(send_invitation_email)

In [ ]:
#| export
def send_password_reset_email(
    to_email: str,
    to_name: str,
    user_name: str,
    reset_url: str,
    test: bool = False
) -> Dict[str, Any]:
    """Send password reset email
    
    Args:
        to_email: User's email address
        to_name: User's full name for email header
        user_name: User's display name for template
        reset_url: Password reset URL with token
        test: If True, print email instead of sending
    
    Returns:
        Result dict from send operation
    """
    return send_email(
        to_email=to_email,
        to_name=to_name,
        subject="Reset Your Password",
        template_name='password_reset',
        template_vars={
            'user_name': user_name,
            'reset_url': reset_url,
            'to_email': to_email
        },
        test=test
    )

In [ ]:
show_doc(send_password_reset_email)